In [125]:
%serialconnect
# PIR pots at top, left: 5V mid: D7=P13  right: G
# looking at PIR pots, left: time, right: sensitivity (clockwise more) 

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB3 --baud=115200 
O[Timed out waiting for recognizable response]
Disconnecting [paste mode not working]
  (You may need to reset the device)

In [91]:
%sendtofile --source ../Sensor_Kennel/BME280_funcs.py
%sendtofile --source ../Sensor_Kennel/TSL561_funcs.py
%sendtofile --quiet --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 79 lines (3304 bytes) to BME280_funcs.py.
Sent 53 lines (1905 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [119]:
%sendtofile config.txt

wifiname     DoESLiverpool
wifipassword decafbad00
mqttbroker   10.0.100.1
pinled       2
pinpir       13
wifialt      BV6000,bubblino,192.168.43.1

Sent 6 lines (147 bytes) to config.txt.


In [5]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pinpir       13
wifialt      BV6000,bubblino,192.168.43.1

Sent 6 lines (144 bytes) to config.txt.


In [126]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [127]:
%sendtofile main.py

import time, sys, machine, itertools
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
from BME280_funcs import bme280init, readBME280
from TSL561_funcs import setupTSL561, setregtimings, luminosity, compute_lux

from stdmqttas import pinled, flashpinled, shortmac
pinpir = machine.Pin(int(fconfig["pinpir"]), machine.Pin.IN)
flashpinled(5, 300, 300)

topicstem = "%s/%s" % (sys.platform, shortmac)
print("topicstem", topicstem)
import network
network.WLAN().active(0)  # disable the connection at startup

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

topicbme280 = topicstem+"/bme280"
topicTSL561 = topicstem+"/TSL561"
topicpir = topicstem+"/pir"
topicstatus = topicstem+"/status"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"

async def bme280task(client):
    try:
        bme280init(i2c, 0x77)
        await asyncio.sleep_ms(200)
        for n in itertools.count():
            meas = readBME280(0x77)
            if (n%100) == 0:
                print("bme", topicbme280, meas)
            if client.isconnected():
                await client.publish(topicbme280, " ".join(map(str, meas)))
            else:
                pinled.value(1-pinled.value())
            await asyncio.sleep_ms(100)
    except OSError as e:
        print("bme280taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(bme280task(client))

        
async def TSL561task(client):
    try:
        setupTSL561(i2c)
        setregtimings(True, 1)
        for n in itertools.count():
            br, ir = luminosity()
            lux = compute_lux(br, ir)
            if (n%200) == 0:
                print("brirlux", topicTSL561, br, ir, lux)
            if client.isconnected():
                await client.publish(topicTSL561, "%d %d %f"%(br, ir, lux))
            await asyncio.sleep_ms(60)
    except OSError as e:
        print("TSL561taskerror", e)
        flashpinled(5, 300, 100)
        aloop.create_task(TSL561task(client))
        
async def pirchangedetectiontask(client):
    pirstate = -2
    for n in itertools.count():
        prevpirstate, pirstate = pirstate, pinpir.value()
        if pirstate != prevpirstate:
            if client.isconnected():
                await client.publish(topicpir, str(pirstate), retain=True)
                blinkstate = ((n%400) == 0)
                pinled.value((1-pirstate) != blinkstate)
            else:
                pirstate = -2
        await asyncio.sleep_ms(5)
        
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
client.DEBUG = True
  
aloop = asyncio.get_event_loop()
aloop.create_task(bme280task(client))
aloop.create_task(TSL561task(client))
aloop.create_task(pirchangedetectiontask(client))
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()


Sent 94 lines (3221 bytes) to main.py.
scandone
state: 0 -> 2 (b0)
state: 2 -> 3 (0)


In [29]:
print(client._sta_if.scan())

[leftinbuffer] ['reconnect']
[leftinbuffer] ['scandone']
[leftinbuffer] ['no DoESLiverpool found, reconnect after 1s']
[leftinbuffer] ['reconnect']
scandone
scandone


Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
OSError: scan failed


In [128]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB10 --baud=115200 
Ready.


In [111]:
#%sendtofile main.py

import time, sys
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from mqtt_as import MQTTClient
import machine
from stdmqttas import slowflashpinled, shortmac

boardname = fconfig["boardname"]    
topicstem = "%s/%s" % (sys.platform, shortmac)
topicbme280 = topicstem+"/bme280"
topicstatus = topicstem+"/status"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    await client.publish(topicstatus, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)
    
def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topicstatus, "offline", True)
client = MQTTClient(config)
    
aloop = asyncio.get_event_loop()
aloop.create_task(mqttconnecttask(client))
aloop.run_forever()



sleep disable
await connecting to: BV6000 192.168.43.1
*** connected:


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 34, in <module>
  File "uasyncio/core.py", line 173, in run_forever
  File "uasyncio/__init__.py", line 69, in wait
KeyboardInterrupt: 
